In [1]:
import pandas as pd
import re
import os
import gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datetime import datetime

# ============================================================================
# 第一部分：DeepSeek 7B 模型載入 (沿用您原有的設定)
# ============================================================================

#MODEL_PATH = "deepseek-ai/deepseek-llm-7b-chat"
MODEL_PATH = "/home/nculcwu/DeepSeek/deepseek-llm-7b-chat"


os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("=== 載入 DeepSeek 7B 模型 ===")
if not torch.cuda.is_available():
    print("錯誤：未偵測到 CUDA。此程式碼需要 NVIDIA GPU 才能執行。")
    exit()

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )
    print("✓ DeepSeek 7B 模型載入成功！")
except Exception as e:
    print(f"模型載入失敗！錯誤訊息: {e}")
    exit()

# ============================================================================
# 第二部分：LLM 輔助函數
# ============================================================================

def query_llm(prompt: str, max_tokens: int = 128) -> str:
    """使用 DeepSeek 7B 執行推理"""
    try:
        messages = [{'role': 'user', 'content': prompt}]
        input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
        attention_mask = torch.ones_like(input_ids)
        outputs = model.generate(
            input_ids.to(model.device),
            attention_mask=attention_mask.to(model.device),
            max_new_tokens=max_tokens,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
        response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        return response.strip()
    except Exception as e:
        print(f"LLM 推理錯誤: {e}")
        return ""

def llm_identify_herbs(text: str) -> dict:
    """
    使用 LLM 識別文本是否為草藥配方
    返回: {"is_herbs": bool, "confidence": str, "reason": str}
    """
    prompt = f"""請分析以下文本是否為中藥草藥配方。請只回答 是 或 否，並簡短說明原因。

文本：「{text}」

格式：
判斷：是/否
原因：[簡短說明]"""

    response = query_llm(prompt, max_tokens=80)
    
    # 解析回應
    is_herbs = False
    confidence = "低"
    reason = "無法判斷"
    
    if "判斷：是" in response or "是" in response[:10]:
        is_herbs = True
        confidence = "高"
    elif "判斷：否" in response or "否" in response[:10]:
        is_herbs = False
        confidence = "高"
    
    # 提取原因
    if "原因：" in response:
        reason = response.split("原因：")[-1].strip()
    elif "因為" in response:
        reason = response.split("因為")[-1].strip()
    else:
        reason = response
    
    return {
        "is_herbs": is_herbs,
        "confidence": confidence,
        "reason": reason[:50]  # 限制長度
    }

def llm_parse_herb_formula(text: str) -> list:
    """
    使用 LLM 解析草藥配方，提取草藥名稱和劑量
    返回: [{"herb": str, "dosage": str}, ...]
    """
    prompt = f"""請解析以下中藥配方，提取每個草藥的名稱和劑量。請按以下格式輸出：

草藥名稱1|劑量1
草藥名稱2|劑量2
...

配方：「{text}」

請注意：
- 如果沒有劑量，劑量欄位留空
- 每行一個草藥
- 使用 | 分隔草藥名稱和劑量"""

    response = query_llm(prompt, max_tokens=200)
    
    herbs = []
    lines = response.strip().split('\n')
    
    for line in lines:
        line = line.strip()
        if '|' in line:
            parts = line.split('|', 1)
            herb_name = parts[0].strip()
            dosage = parts[1].strip() if len(parts) > 1 else ""
            
            if herb_name and len(herb_name) <= 10:  # 草藥名稱通常不會太長
                herbs.append({
                    "herb": herb_name,
                    "dosage": dosage
                })
    
    return herbs

def llm_classify_herb_content(text: str) -> dict:
    """
    使用 LLM 對文本內容進行分類：草藥配方 vs 疾病描述
    """
    prompt = f"""請分析以下文本內容，判斷它是「草藥配方」還是「疾病描述」。

文本：「{text}」

請只回答：
類別：草藥配方/疾病描述
信心度：高/中/低"""

    response = query_llm(prompt, max_tokens=50)
    
    category = "未知"
    confidence = "低"
    
    if "草藥配方" in response:
        category = "草藥配方"
    elif "疾病描述" in response:
        category = "疾病描述"
    
    if "信心度：高" in response:
        confidence = "高"
    elif "信心度：中" in response:
        confidence = "中"
    
    return {
        "category": category,
        "confidence": confidence
    }

# ============================================================================
# 第三部分：結合 LLM 的智能預處理
# ============================================================================

def smart_preprocess_with_llm(df, ingredients_column='ingredients', disease_column='disease'):
    """
    使用 LLM 進行智能預處理，比傳統正規表達式更準確
    """
    print("=== 執行 LLM 智能預處理 ===")
    
    processed_df = df.copy()
    total_rows = len(df)
    empty_ingredients_count = 0
    transferred_count = 0
    llm_decisions = []
    
    for index, row in processed_df.iterrows():
        print(f"處理第 {index+1}/{total_rows} 行...", end=" ")
        
        ingredients_value = row[ingredients_column] if ingredients_column in row else None
        disease_value = row[disease_column] if disease_column in row else None
        
        # 檢查 ingredients 是否為空
        is_ingredients_empty = (
            pd.isna(ingredients_value) or 
            str(ingredients_value).strip() == '' or 
            str(ingredients_value) == 'nan'
        )
        
        if is_ingredients_empty:
            empty_ingredients_count += 1
            
            # 檢查 disease 欄位是否有內容
            if not pd.isna(disease_value) and str(disease_value).strip():
                disease_text = str(disease_value).strip()
                
                # 使用 LLM 判斷是否為草藥配方
                llm_result = llm_classify_herb_content(disease_text)
                llm_decisions.append({
                    'row': index + 1,
                    'text': disease_text[:30] + '...' if len(disease_text) > 30 else disease_text,
                    'decision': llm_result
                })
                
                if llm_result['category'] == '草藥配方' and llm_result['confidence'] in ['高', '中']:
                    # 轉移資料
                    processed_df.at[index, ingredients_column] = disease_text
                    transferred_count += 1
                    print(f"✓ 轉移 (信心度: {llm_result['confidence']})")
                else:
                    print(f"✗ 跳過 ({llm_result['category']}, 信心度: {llm_result['confidence']})")
            else:
                print("✗ disease 欄位為空")
        else:
            print("✓ ingredients 已有內容")
        
        # 清理 GPU 記憶體
        if index % 10 == 0:
            gc.collect()
            torch.cuda.empty_cache()
    
    print(f"\n=== LLM 預處理統計 ===")
    print(f"總行數: {total_rows}")
    print(f"ingredients 為空的行數: {empty_ingredients_count}")
    print(f"LLM 成功轉移的行數: {transferred_count}")
    
    if llm_decisions:
        print(f"\nLLM 決策詳情:")
        for decision in llm_decisions:
            print(f"  第 {decision['row']} 行: {decision['text']} → {decision['decision']['category']} (信心度: {decision['decision']['confidence']})")
    
    return processed_df, {
        'total_rows': total_rows,
        'empty_ingredients': empty_ingredients_count,
        'transferred': transferred_count,
        'llm_decisions': llm_decisions
    }

# ============================================================================
# 第四部分：LLM 輔助的草藥拆分
# ============================================================================

def smart_split_ingredients_with_llm(df, ingredients_column='ingredients', use_llm_parsing=True):
    """
    結合傳統正規表達式和 LLM 的草藥拆分
    """
    print("=== 執行 LLM 輔助草藥拆分 ===")
    
    expanded_rows = []
    total_rows = len(df)
    llm_parsing_count = 0
    regex_parsing_count = 0
    
    for index, row in df.iterrows():
        print(f"拆分第 {index+1}/{total_rows} 行...", end=" ")
        
        ingredients_text = str(row[ingredients_column])
        
        # 如果該欄位為空或無效，保留原行但新增空的草藥欄位
        if pd.isna(row[ingredients_column]) or ingredients_text.strip() == '' or ingredients_text == 'nan':
            new_row = row.copy()
            new_row['草藥'] = ''
            new_row['劑量說明'] = ''
            new_row['解析方式'] = '空白'
            expanded_rows.append(new_row)
            print("空白內容")
            continue
        
        # 首先嘗試傳統正規表達式解析
        herb_pattern = r'([^〈〉<>\s]+)\s*(?:〈([^〈〉]*)〉|<([^<>]*)>)?'
        cleaned_text = re.sub(r'\s+', ' ', ingredients_text.strip())
        regex_matches = re.findall(herb_pattern, cleaned_text)
        
        parsing_method = "正規表達式"
        herbs_data = []
        
        if regex_matches and len(regex_matches) > 1:  # 正規表達式成功解析出多個草藥
            for match in regex_matches:
                herb_name = match[0].strip()
                dosage = (match[1] or match[2] or '').strip()
                if herb_name:
                    herbs_data.append({"herb": herb_name, "dosage": dosage})
            regex_parsing_count += 1
            print(f"正規表達式解析 ({len(herbs_data)} 個草藥)")
        
        elif use_llm_parsing:  # 正規表達式失效時使用 LLM
            llm_herbs = llm_parse_herb_formula(ingredients_text)
            if llm_herbs:
                herbs_data = llm_herbs
                parsing_method = "LLM解析"
                llm_parsing_count += 1
                print(f"LLM解析 ({len(herbs_data)} 個草藥)")
            else:
                # LLM 也失敗時，保留原文
                herbs_data = [{"herb": ingredients_text, "dosage": ""}]
                parsing_method = "原文保留"
                print("原文保留")
        else:
            # 不使用 LLM 時的備用方案
            herbs_data = [{"herb": ingredients_text, "dosage": ""}]
            parsing_method = "原文保留"
            print("原文保留")
        
        # 為每個草藥創建一行
        for herb_data in herbs_data:
            new_row = row.copy()
            new_row['草藥'] = herb_data['herb']
            new_row['劑量說明'] = herb_data['dosage']
            new_row['解析方式'] = parsing_method
            expanded_rows.append(new_row)
        
        # 定期清理記憶體
        if index % 5 == 0:
            gc.collect()
            torch.cuda.empty_cache()
    
    # 轉換為 DataFrame
    result_df = pd.DataFrame(expanded_rows)
    
    # 重新排序欄位
    cols = result_df.columns.tolist()
    if '草藥' in cols and '劑量說明' in cols and '解析方式' in cols:
        for col in ['草藥', '劑量說明', '解析方式']:
            cols.remove(col)
        new_cols = ['草藥', '劑量說明', '解析方式'] + cols
        result_df = result_df[new_cols]
    
    print(f"\n=== 拆分統計 ===")
    print(f"總計處理: {total_rows} 行")
    print(f"正規表達式成功: {regex_parsing_count} 行")
    print(f"LLM 解析成功: {llm_parsing_count} 行")
    print(f"最終草藥條目: {len(result_df)} 行")
    
    return result_df

# ============================================================================
# 第五部分：主執行流程
# ============================================================================

def main_process_with_llm():
    """
    主處理流程：整合 LLM 的智能草藥處理
    """
    # --- 使用者設定區 ---
    FOLDER_PATH = '.'
    FILE_PATTERN = 'classified_section_卷'
    INGREDIENTS_COLUMN = 'ingredients'
    USE_LLM_PARSING = True  # 是否使用 LLM 輔助解析
    
    print("=== LLM 增強草藥處理系統啟動 ===")
    print(f"掃描資料夾: {os.path.abspath(FOLDER_PATH)}")
    print(f"LLM 輔助解析: {'啟用' if USE_LLM_PARSING else '停用'}")
    
    # 掃描目標檔案
    target_files = []
    for filename in os.listdir(FOLDER_PATH):
        if filename.startswith(FILE_PATTERN) and filename.endswith('.xlsx'):
            target_files.append(filename)
    
    if not target_files:
        print(f"錯誤：找不到符合 '{FILE_PATTERN}*.xlsx' 格式的檔案。")
        return
    
    target_files.sort()
    print(f"找到 {len(target_files)} 個目標檔案")
    
    # 處理檔案
    for filename in target_files:
        file_path = os.path.join(FOLDER_PATH, filename)
        print(f"\n{'='*50}")
        print(f"正在處理: {filename}")
        print(f"{'='*50}")
        
        try:
            # 讀取檔案
            df = pd.read_excel(file_path)
            print(f"原始資料: {len(df)} 行, {len(df.columns)} 欄")
            
            # 檢查必要欄位
            if INGREDIENTS_COLUMN not in df.columns:
                print(f"跳過：找不到 '{INGREDIENTS_COLUMN}' 欄位")
                continue
            
            # 步驟 1: LLM 智能預處理
            if 'disease' in df.columns:
                df, preprocess_stats = smart_preprocess_with_llm(df, INGREDIENTS_COLUMN, 'disease')
            else:
                print("跳過預處理：無 disease 欄位")
            
            # 步驟 2: LLM 輔助草藥拆分
            expanded_df = smart_split_ingredients_with_llm(df, INGREDIENTS_COLUMN, USE_LLM_PARSING)
            
            # 步驟 3: 儲存結果
            base_name = filename.replace('.xlsx', '')
            today_str = datetime.now().strftime("%Y%m%d_%H%M")
            output_filename = f"{base_name}_LLM智能拆分_{today_str}.xlsx"
            output_path = os.path.join(FOLDER_PATH, output_filename)
            
            expanded_df.to_excel(output_path, index=False)
            print(f"\n✓ 成功儲存: {output_filename}")
            print(f"  最終結果: {len(expanded_df)} 行, {len(expanded_df.columns)} 欄")
            
            # 顯示解析方式統計
            if '解析方式' in expanded_df.columns:
                parsing_stats = expanded_df['解析方式'].value_counts()
                print("  解析方式統計:")
                for method, count in parsing_stats.items():
                    print(f"    {method}: {count} 條")
            
        except Exception as e:
            print(f"✗ 處理失敗: {e}")
            continue
    
    print(f"\n{'='*50}")
    print("所有處理完成！")

# ============================================================================
# 第六部分：示範模式
# ============================================================================

def demo_llm_herb_processing():
    """
    示範 LLM 草藥處理功能
    """
    print("=== LLM 草藥處理示範 ===")
    
    # 測試資料
    test_data = [
        "草豆 黒牽牛〈各二兩〉糖毬〈四兩〉木香",
        "當歸 川芎 白芍 甘草各等分",
        "人參三錢 黃耆五錢 甘草一錢半",
        "",
        "頭痛眩暈，面色蒼白，心悸氣短"
    ]
    
    # 創建測試 DataFrame
    test_df = pd.DataFrame({
        'id': range(1, len(test_data) + 1),
        'ingredients': ["", "", "", "", ""],  # ingredients 故意留空
        'disease': test_data  # 資料在 disease 欄位
    })
    
    print("原始測試資料:")
    print(test_df)
    
    # 執行 LLM 預處理
    processed_df, stats = smart_preprocess_with_llm(test_df)
    
    print("\n預處理後:")
    print(processed_df[['id', 'ingredients', 'disease']])
    
    # 執行拆分
    final_df = smart_split_ingredients_with_llm(processed_df, use_llm_parsing=True)
    
    print("\n最終結果:")
    print(final_df[['草藥', '劑量說明', '解析方式', 'id']])
    
    # 儲存示範結果
    today_str = datetime.now().strftime("%Y%m%d_%H%M")
    demo_filename = f"LLM草藥處理示範_{today_str}.xlsx"
    final_df.to_excel(demo_filename, index=False)
    print(f"\n✓ 示範結果已儲存: {demo_filename}")

if __name__ == "__main__":
    print("請選擇執行模式:")
    print("1. 處理實際檔案 (使用 LLM)")
    print("2. 示範 LLM 功能")
    print("3. 傳統模式 (不使用 LLM)")
    
    choice = input("請輸入選項 (1/2/3): ").strip()
    
    if choice == "1":
        main_process_with_llm()
    elif choice == "2":
        demo_llm_herb_processing()
    elif choice == "3":
        # 使用原本的非 LLM 版本
        from original_herb_splitter import process_files_with_ingredient_splitting
        process_files_with_ingredient_splitting()
    else:
        print("預設執行示範模式...")
        demo_llm_herb_processing()

/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== 載入 DeepSeek 7B 模型 ===


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.85s/it]

✓ DeepSeek 7B 模型載入成功！
請選擇執行模式:
1. 處理實際檔案 (使用 LLM)
2. 示範 LLM 功能
3. 傳統模式 (不使用 LLM)


=== LLM 草藥處理示範 ===
原始測試資料:
   id ingredients              disease
0   1              草豆 黒牽牛〈各二兩〉糖毬〈四兩〉木香
1   2                   當歸 川芎 白芍 甘草各等分
2   3                  人參三錢 黃耆五錢 甘草一錢半
3   4                                 
4   5                   頭痛眩暈，面色蒼白，心悸氣短
=== 執行 LLM 智能預處理 ===
處理第 1/5 行... 

/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


✓ 轉移 (信心度: 高)
處理第 2/5 行... ✓ 轉移 (信心度: 高)
處理第 3/5 行... ✓ 轉移 (信心度: 高)
處理第 4/5 行... ✗ disease 欄位為空
處理第 5/5 行... ✓ 轉移 (信心度: 高)

=== LLM 預處理統計 ===
總行數: 5
ingredients 為空的行數: 5
LLM 成功轉移的行數: 4

LLM 決策詳情:
  第 1 行: 草豆 黒牽牛〈各二兩〉糖毬〈四兩〉木香 → 草藥配方 (信心度: 高)
  第 2 行: 當歸 川芎 白芍 甘草各等分 → 草藥配方 (信心度: 高)
  第 3 行: 人參三錢 黃耆五錢 甘草一錢半 → 草藥配方 (信心度: 高)
  第 5 行: 頭痛眩暈，面色蒼白，心悸氣短 → 草藥配方 (信心度: 高)

預處理後:
   id          ingredients              disease
0   1  草豆 黒牽牛〈各二兩〉糖毬〈四兩〉木香  草豆 黒牽牛〈各二兩〉糖毬〈四兩〉木香
1   2       當歸 川芎 白芍 甘草各等分       當歸 川芎 白芍 甘草各等分
2   3      人參三錢 黃耆五錢 甘草一錢半      人參三錢 黃耆五錢 甘草一錢半
3   4                                          
4   5       頭痛眩暈，面色蒼白，心悸氣短       頭痛眩暈，面色蒼白，心悸氣短
=== 執行 LLM 輔助草藥拆分 ===
拆分第 1/5 行... 正規表達式解析 (4 個草藥)
拆分第 2/5 行... 正規表達式解析 (4 個草藥)
拆分第 3/5 行... 正規表達式解析 (3 個草藥)
拆分第 4/5 行... 空白內容
拆分第 5/5 行... 

/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


LLM解析 (5 個草藥)

=== 拆分統計 ===
總計處理: 5 行
正規表達式成功: 3 行
LLM 解析成功: 1 行
最終草藥條目: 17 行

最終結果:
      草藥 劑量說明   解析方式  id
0     草豆       正規表達式   1
0    黒牽牛  各二兩  正規表達式   1
0     糖毬   四兩  正規表達式   1
0     木香       正規表達式   1
1     當歸       正規表達式   2
1     川芎       正規表達式   2
1     白芍       正規表達式   2
1  甘草各等分       正規表達式   2
2   人參三錢       正規表達式   3
2   黃耆五錢       正規表達式   3
2  甘草一錢半       正規表達式   3
3                 空白   4
4  草藥名稱1  劑量1  LLM解析   5
4  草藥名稱2  劑量2  LLM解析   5
4  草藥名稱3  劑量3  LLM解析   5
4  草藥名稱4  劑量4  LLM解析   5
4  草藥名稱5  劑量5  LLM解析   5

✓ 示範結果已儲存: LLM草藥處理示範_20250827_1237.xlsx


優化板

In [2]:
import pandas as pd
import re
import os
import gc
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datetime import datetime
import time

# ============================================================================
# 第一部分：DeepSeek 7B 模型載入
# ============================================================================

MODEL_PATH = "/home/nculcwu/DeepSeek/deepseek-llm-7b-chat"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

print("=== 載入 DeepSeek 7B 模型 ===")
if not torch.cuda.is_available():
    print("錯誤：未偵測到 CUDA。此程式碼需要 NVIDIA GPU 才能執行。")
    exit()

try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.float16,
        device_map="auto",
        low_cpu_mem_usage=True,
        trust_remote_code=True
    )
    print("✓ DeepSeek 7B 模型載入成功！")
except Exception as e:
    print(f"模型載入失敗！錯誤訊息: {e}")
    exit()

# ============================================================================
# 第二部分：優化的LLM輔助函數
# ============================================================================

def query_llm_with_timeout(prompt: str, max_tokens: int = 128, timeout_seconds: int = 30) -> str:
    """使用 DeepSeek 7B 執行推理，加入超時控制"""
    start_time = time.time()
    try:
        messages = [{'role': 'user', 'content': prompt}]
        input_ids = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
        attention_mask = torch.ones_like(input_ids)
        
        # 檢查輸入長度
        if input_ids.shape[1] > 2048:  # 如果輸入太長
            print(f"    警告：輸入過長 ({input_ids.shape[1]} tokens)，跳過LLM處理")
            return ""
        
        outputs = model.generate(
            input_ids.to(model.device),
            attention_mask=attention_mask.to(model.device),
            max_new_tokens=max_tokens,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id
        )
        
        response = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        elapsed_time = time.time() - start_time
        print(f" ({elapsed_time:.1f}s)", end="")
        return response.strip()
        
    except Exception as e:
        elapsed_time = time.time() - start_time
        print(f"    LLM 推理錯誤 ({elapsed_time:.1f}s): {e}")
        return ""

def fast_preprocess_check(text: str) -> bool:
    """
    快速預檢查：使用簡單規則判斷是否需要LLM處理
    如果明顯不是草藥配方，就不調用LLM
    """
    if not text or len(text.strip()) < 3:
        return False
    
    # 草藥配方的特徵
    herb_indicators = [
        r'[〈<].*[〉>]',  # 包含劑量標記
        r'(錢|兩|克|g|分|厘|斤)',  # 計量單位
        r'(各|共|君|臣|佐|使|等分)',  # 中醫術語
        r'[\u4e00-\u9fa5]{2,4}[\s〈<]',  # 中文名稱後跟空格或括號
    ]
    
    # 疾病描述的特徵
    disease_indicators = [
        r'(痛|痠|脹|悶|癢|熱|寒|虛|實)',  # 症狀詞
        r'(頭|胸|腹|背|腰|四肢)',  # 身體部位
        r'(患|病|症|候|證)',  # 疾病相關詞
        r'(面色|心悸|氣短|眩暈)',  # 症狀描述
    ]
    
    herb_score = sum(1 for pattern in herb_indicators if re.search(pattern, text))
    disease_score = sum(1 for pattern in disease_indicators if re.search(pattern, text))
    
    # 如果明顯是疾病描述，不需要LLM
    if disease_score > herb_score and disease_score >= 2:
        return False
    
    # 如果有草藥特徵或不確定，需要LLM
    return herb_score > 0 or disease_score == 0

def llm_classify_herb_content_fast(text: str) -> dict:
    """優化版的內容分類，加入快速預檢查"""
    
    # 快速預檢查
    if not fast_preprocess_check(text):
        return {
            "category": "疾病描述",
            "confidence": "中",
            "method": "規則判斷"
        }
    
    # 需要LLM判斷的情況
    prompt = f"""請快速判斷以下文本是「草藥配方」還是「疾病描述」。只回答分類結果。

文本：「{text}」

回答格式：草藥配方/疾病描述"""

    response = query_llm_with_timeout(prompt, max_tokens=20)
    
    category = "未知"
    confidence = "中"
    
    if "草藥配方" in response:
        category = "草藥配方"
        confidence = "高"
    elif "疾病描述" in response:
        category = "疾病描述" 
        confidence = "高"
    else:
        # LLM無法判斷時，使用預設規則
        category = "草藥配方" if fast_preprocess_check(text) else "疾病描述"
        confidence = "低"
    
    return {
        "category": category,
        "confidence": confidence,
        "method": "LLM判斷"
    }

# ============================================================================
# 第三部分：批次處理和進度監控
# ============================================================================

def smart_preprocess_with_progress(df, ingredients_column='ingredients', disease_column='disease', max_llm_calls=50):
    """
    加入進度監控和限制的智能預處理
    max_llm_calls: 限制LLM調用次數，避免處理時間過長
    """
    print("=== 執行優化版智能預處理 ===")
    
    processed_df = df.copy()
    total_rows = len(df)
    empty_ingredients_count = 0
    transferred_count = 0
    llm_call_count = 0
    skipped_count = 0
    
    print(f"總行數: {total_rows}，LLM調用上限: {max_llm_calls}")
    
    for index, row in processed_df.iterrows():
        # 進度顯示
        if index % 10 == 0:
            print(f"\n進度: {index}/{total_rows} ({100*index/total_rows:.1f}%)")
        
        print(f"  第 {index+1} 行:", end=" ")
        
        ingredients_value = row[ingredients_column] if ingredients_column in row else None
        disease_value = row[disease_column] if disease_column in row else None
        
        # 檢查 ingredients 是否為空
        is_ingredients_empty = (
            pd.isna(ingredients_value) or 
            str(ingredients_value).strip() == '' or 
            str(ingredients_value) == 'nan'
        )
        
        if is_ingredients_empty:
            empty_ingredients_count += 1
            
            if not pd.isna(disease_value) and str(disease_value).strip():
                disease_text = str(disease_value).strip()
                
                # 限制LLM調用次數
                if llm_call_count >= max_llm_calls:
                    print("達到LLM調用上限，跳過")
                    skipped_count += 1
                    continue
                
                # 使用優化版分類
                llm_result = llm_classify_herb_content_fast(disease_text)
                llm_call_count += 1
                
                if llm_result['category'] == '草藥配方' and llm_result['confidence'] in ['高', '中']:
                    processed_df.at[index, ingredients_column] = disease_text
                    transferred_count += 1
                    print(f"✓ 轉移")
                else:
                    print(f"✗ 跳過 ({llm_result['category']})")
            else:
                print("disease欄位空白")
        else:
            print("ingredients已有內容")
        
        # 記憶體清理
        if index % 20 == 0:
            gc.collect()
            torch.cuda.empty_cache()
    
    print(f"\n=== 預處理統計 ===")
    print(f"總行數: {total_rows}")
    print(f"ingredients為空: {empty_ingredients_count}")
    print(f"成功轉移: {transferred_count}")
    print(f"LLM調用次數: {llm_call_count}")
    print(f"跳過處理: {skipped_count}")
    
    return processed_df, {
        'total_rows': total_rows,
        'empty_ingredients': empty_ingredients_count,
        'transferred': transferred_count,
        'llm_calls': llm_call_count,
        'skipped': skipped_count
    }

def optimized_split_ingredients(df, ingredients_column='ingredients', use_llm_limit=20):
    """
    優化版草藥拆分，限制LLM使用次數
    """
    print("=== 執行優化版草藥拆分 ===")
    
    expanded_rows = []
    total_rows = len(df)
    llm_parsing_count = 0
    regex_parsing_count = 0
    original_kept_count = 0
    
    print(f"開始處理 {total_rows} 行，LLM解析上限: {use_llm_limit}")
    
    for index, row in df.iterrows():
        # 進度顯示
        if index % 20 == 0:
            print(f"\n拆分進度: {index}/{total_rows} ({100*index/total_rows:.1f}%)")
        
        print(f"  第 {index+1} 行:", end=" ")
        
        ingredients_text = str(row[ingredients_column])
        
        # 空白處理
        if pd.isna(row[ingredients_column]) or ingredients_text.strip() == '' or ingredients_text == 'nan':
            new_row = row.copy()
            new_row['草藥'] = ''
            new_row['劑量說明'] = ''
            new_row['解析方式'] = '空白'
            expanded_rows.append(new_row)
            print("空白")
            continue
        
        # 傳統正規表達式解析
        herb_pattern = r'([^〈〉<>\s]+)\s*(?:〈([^〈〉]*)〉|<([^<>]*)>)?'
        cleaned_text = re.sub(r'\s+', ' ', ingredients_text.strip())
        regex_matches = re.findall(herb_pattern, cleaned_text)
        
        herbs_data = []
        parsing_method = "原文保留"
        
        # 檢查正規表達式結果
        if regex_matches and len(regex_matches) >= 2:  # 至少要有2個草藥才算成功
            valid_herbs = []
            for match in regex_matches:
                herb_name = match[0].strip()
                dosage = (match[1] or match[2] or '').strip()
                if herb_name and len(herb_name) <= 8:  # 草藥名稱長度合理
                    valid_herbs.append({"herb": herb_name, "dosage": dosage})
            
            if len(valid_herbs) >= 2:
                herbs_data = valid_herbs
                parsing_method = "正規表達式"
                regex_parsing_count += 1
                print(f"正規表達式 ({len(herbs_data)}個)")
        
        # 如果正規表達式失敗且未達LLM使用上限
        if not herbs_data and llm_parsing_count < use_llm_limit:
            if len(ingredients_text) < 100:  # 只對短文本使用LLM
                print("嘗試LLM解析...", end="")
                llm_herbs = llm_parse_herb_formula_simple(ingredients_text)
                if llm_herbs and len(llm_herbs) >= 1:
                    herbs_data = llm_herbs
                    parsing_method = "LLM解析"
                    llm_parsing_count += 1
                    print(f"成功 ({len(herbs_data)}個)")
                else:
                    print("失敗")
            else:
                print("文本過長，跳過LLM")
        
        # 最終處理
        if not herbs_data:
            herbs_data = [{"herb": ingredients_text, "dosage": ""}]
            original_kept_count += 1
            print("保留原文")
        
        # 創建結果行
        for herb_data in herbs_data:
            new_row = row.copy()
            new_row['草藥'] = herb_data['herb']
            new_row['劑量說明'] = herb_data['dosage']
            new_row['解析方式'] = parsing_method
            expanded_rows.append(new_row)
        
        # 記憶體清理
        if index % 15 == 0:
            gc.collect()
            torch.cuda.empty_cache()
    
    print(f"\n=== 拆分統計 ===")
    print(f"總計處理: {total_rows} 行")
    print(f"正規表達式成功: {regex_parsing_count} 行")
    print(f"LLM解析成功: {llm_parsing_count} 行")
    print(f"保留原文: {original_kept_count} 行")
    print(f"最終草藥條目: {len(expanded_rows)} 行")
    
    # 轉換為DataFrame並重新排序
    result_df = pd.DataFrame(expanded_rows)
    cols = result_df.columns.tolist()
    if '草藥' in cols and '劑量說明' in cols and '解析方式' in cols:
        for col in ['草藥', '劑量說明', '解析方式']:
            cols.remove(col)
        new_cols = ['草藥', '劑量說明', '解析方式'] + cols
        result_df = result_df[new_cols]
    
    return result_df

def llm_parse_herb_formula_simple(text: str) -> list:
    """簡化版的LLM草藥解析"""
    prompt = f"""解析中藥配方，列出草藥名稱和劑量：

{text}

格式：草藥|劑量
每行一個草藥"""

    response = query_llm_with_timeout(prompt, max_tokens=150)
    
    herbs = []
    lines = response.strip().split('\n')
    
    for line in lines[:10]:  # 限制最多10個草藥
        line = line.strip()
        if '|' in line:
            parts = line.split('|', 1)
            herb_name = parts[0].strip()
            dosage = parts[1].strip() if len(parts) > 1 else ""
            
            if herb_name and len(herb_name) <= 8:
                herbs.append({"herb": herb_name, "dosage": dosage})
    
    return herbs

# ============================================================================
# 主執行流程 - 優化版
# ============================================================================

def main_process_optimized():
    """
    優化版主處理流程
    """
    # 設定參數
    FOLDER_PATH = '.'
    FILE_PATTERN = 'classified_section_卷'
    INGREDIENTS_COLUMN = 'ingredients'
    MAX_LLM_PREPROCESS = 30  # 預處理階段最多30次LLM調用
    MAX_LLM_PARSING = 20     # 解析階段最多20次LLM調用
    
    print("=== 優化版LLM草藥處理系統 ===")
    print(f"LLM調用限制 - 預處理: {MAX_LLM_PREPROCESS}, 解析: {MAX_LLM_PARSING}")
    
    # 掃描檔案
    target_files = [f for f in os.listdir(FOLDER_PATH) 
                   if f.startswith(FILE_PATTERN) and f.endswith('.xlsx')]
    
    if not target_files:
        print("未找到目標檔案")
        return
    
    target_files.sort()
    print(f"找到 {len(target_files)} 個檔案: {target_files}")
    
    # 處理檔案
    for filename in target_files:
        print(f"\n{'='*60}")
        print(f"處理檔案: {filename}")
        print(f"{'='*60}")
        
        start_time = time.time()
        
        try:
            # 讀取檔案
            file_path = os.path.join(FOLDER_PATH, filename)
            df = pd.read_excel(file_path)
            print(f"讀取完成: {len(df)} 行, {len(df.columns)} 欄")
            
            if INGREDIENTS_COLUMN not in df.columns:
                print(f"跳過: 無 {INGREDIENTS_COLUMN} 欄位")
                continue
            
            # 預處理
            if 'disease' in df.columns:
                print(f"\n--- 步驟1: 預處理 (上限 {MAX_LLM_PREPROCESS} 次LLM調用) ---")
                df, stats = smart_preprocess_with_progress(df, INGREDIENTS_COLUMN, 'disease', MAX_LLM_PREPROCESS)
            else:
                print("跳過預處理: 無disease欄位")
            
            # 拆分
            print(f"\n--- 步驟2: 草藥拆分 (上限 {MAX_LLM_PARSING} 次LLM調用) ---")
            result_df = optimized_split_ingredients(df, INGREDIENTS_COLUMN, MAX_LLM_PARSING)
            
            # 儲存
            base_name = filename.replace('.xlsx', '')
            timestamp = datetime.now().strftime("%m%d_%H%M")
            output_filename = f"{base_name}_優化拆分_{timestamp}.xlsx"
            output_path = os.path.join(FOLDER_PATH, output_filename)
            
            result_df.to_excel(output_path, index=False)
            
            elapsed_time = time.time() - start_time
            print(f"\n✓ 完成: {output_filename}")
            print(f"  處理時間: {elapsed_time:.1f} 秒")
            print(f"  結果: {len(result_df)} 行")
            
            # 最終統計
            if '解析方式' in result_df.columns:
                method_stats = result_df['解析方式'].value_counts()
                print("  解析統計:")
                for method, count in method_stats.items():
                    print(f"    {method}: {count}")
            
        except Exception as e:
            print(f"✗ 處理失敗: {e}")
            continue
        
        print(f"\n處理 {filename} 完成，總用時: {time.time() - start_time:.1f} 秒")
    
    print("\n" + "="*60)
    print("所有檔案處理完成！")

# ============================================================================
# 示範模式保持不變
# ============================================================================

def demo_llm_herb_processing():
    """示範功能"""
    print("=== LLM草藥處理示範 ===")
    
    test_data = [
        "草豆 黒牽牛〈各二兩〉糖毬〈四兩〉木香",
        "當歸 川芎 白芍 甘草各等分", 
        "人參三錢 黃耆五錢 甘草一錢半",
        "",
        "頭痛眩暈，面色蒼白，心悸氣短"
    ]
    
    test_df = pd.DataFrame({
        'id': range(1, len(test_data) + 1),
        'ingredients': ["", "", "", "", ""],
        'disease': test_data
    })
    
    print("原始資料:")
    print(test_df)
    
    # 預處理
    processed_df, stats = smart_preprocess_with_progress(test_df, max_llm_calls=10)
    print("\n預處理後:")
    print(processed_df[['id', 'ingredients', 'disease']])
    
    # 拆分
    final_df = optimized_split_ingredients(processed_df, use_llm_limit=5)
    print("\n最終結果:")
    print(final_df[['草藥', '劑量說明', '解析方式', 'id']])
    
    # 儲存
    timestamp = datetime.now().strftime("%m%d_%H%M") 
    demo_filename = f"優化版示範_{timestamp}.xlsx"
    final_df.to_excel(demo_filename, index=False)
    print(f"\n✓ 儲存: {demo_filename}")

if __name__ == "__main__":
    print("優化版LLM草藥處理系統")
    print("1. 處理實際檔案 (優化版)")
    print("2. 示範功能")
    
    choice = input("選擇 (1/2): ").strip()
    
    if choice == "1":
        main_process_optimized()
    else:
        demo_llm_herb_processing()

=== 載入 DeepSeek 7B 模型 ===


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.83s/it]

✓ DeepSeek 7B 模型載入成功！
優化版LLM草藥處理系統
1. 處理實際檔案 (優化版)
2. 示範功能


=== 優化版LLM草藥處理系統 ===
LLM調用限制 - 預處理: 30, 解析: 20
找到 5 個檔案: ['classified_section_卷169.xlsx', 'classified_section_卷169_LLM智能拆分_20250827_1217.xlsx', 'classified_section_卷169_LLM智能拆分_20250827_1217_LLM智能拆分_20250827_1236.xlsx', 'classified_section_卷169_LLM智能拆分_20250827_1230.xlsx', 'classified_section_卷169_LLM智能拆分_20250827_1233.xlsx']

處理檔案: classified_section_卷169.xlsx
讀取完成: 90 行, 8 欄

--- 步驟1: 預處理 (上限 30 次LLM調用) ---
=== 執行優化版智能預處理 ===
總行數: 90，LLM調用上限: 30

進度: 0/90 (0.0%)
  第 1 行: ingredients已有內容
  第 2 行: ingredients已有內容
  第 3 行: ingredients已有內容
  第 4 行: ingredients已有內容
  第 5 行: ingredients已有內容
  第 6 行: ingredients已有內容
  第 7 行: ingredients已有內容
  第 8 行: ingredients已有內容
  第 9 行: ingredients已有內容
  第 10 行: ingredients已有內容

進度: 10/90 (11.1%)
  第 11 行: ingredients已有內容
  第 12 行: ingredients已有內容
  第 13 行: ingredients已有內容
  第 14 行: ingredients已有內容
  第 15 行: ingredients已有內容
  第 16 行: ingredients已有內容
  第 17 行: ingredients已有內容
  第 18 行: ingredients已有內容
  第 19 行: ingredients已有內容
  第 20 行: ingredients已有內容



/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


...

 (4.2s)成功 (1個)
  第 1236 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1237 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1238 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1239 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1240 行: 嘗試LLM解析... (4.2s)成功 (1個)

拆分進度: 1240/8808 (14.1%)
  第 1241 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1242 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1243 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1244 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1245 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1246 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1247 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1248 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1249 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1250 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1251 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1252 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1253 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1254 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 1255 行: 保留原文
  第 1256 行: 保留原文
  第 1257 行: 保留原文
  第 1258 行: 保留原文
  第 1259 行: 保留原文
  第 1260 行: 保留原文

拆分進度: 1260/8808 (14.3%)
  第 1261 行: 保留原文
  第 1262 行: 保留原文
  第 1263 行: 保留原文
  第 1264 行: 保留原文
  第 1265 行: 保留原文
  第 1266 行: 保留原文
  第 1267 行: 保留原文
  第 1268 行:

/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 (4.3s)成功 (1個)
  第 140 行: 嘗試LLM解析... (4.2s)成功 (1個)

拆分進度: 140/714 (19.6%)
  第 141 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 142 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 143 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 144 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 145 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 146 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 147 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 148 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 149 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 150 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 151 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 152 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 153 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 154 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 155 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 156 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 157 行: 嘗試LLM解析... (4.3s)失敗
保留原文
  第 158 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 159 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 160 行: 保留原文

拆分進度: 160/714 (22.4%)
  第 161 行: 保留原文
  第 162 行: 保留原文
  第 163 行: 正規表達式 (4個)
  第 164 行: 正規表達式 (4個)
  第 165 行: 正規表達式 (4個)
  第 166 行: 正規表達式 (4個)
  第 167 行: 保留原文
  第 168 行: 保留原文
  第 169 行: 保留原文
  第 170 行: 正規表達式 (6個)
  第 171 行: 正規

/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/nculcwu/DeepSeek/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


 (4.3s)成功 (1個)
  第 140 行: 嘗試LLM解析... (4.2s)成功 (1個)

拆分進度: 140/714 (19.6%)
  第 141 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 142 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 143 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 144 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 145 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 146 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 147 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 148 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 149 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 150 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 151 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 152 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 153 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 154 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 155 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 156 行: 嘗試LLM解析... (4.2s)成功 (1個)
  第 157 行: 嘗試LLM解析... (4.2s)失敗
保留原文
  第 158 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 159 行: 嘗試LLM解析... (4.3s)成功 (1個)
  第 160 行: 保留原文

拆分進度: 160/714 (22.4%)
  第 161 行: 保留原文
  第 162 行: 保留原文
  第 163 行: 正規表達式 (4個)
  第 164 行: 正規表達式 (4個)
  第 165 行: 正規表達式 (4個)
  第 166 行: 正規表達式 (4個)
  第 167 行: 保留原文
  第 168 行: 保留原文
  第 169 行: 保留原文
  第 170 行: 正規表達式 (6個)
  第 171 行: 正規